In [62]:
import pymorphy2
import nltk
import string
from nltk.corpus import stopwords


morph = pymorphy2.MorphAnalyzer()

stop_words = stopwords.words('russian')
stop_words.extend(['«', '»', '–', '...', '“', '”', '—', '!',
                   '@', '№', ':', ',', '.', '?', ':', '(', ')'])
stop_words.extend(['который', 'каждый'])
stop_words = set(stop_words)


def normalize_review(review):
    """

    :param review:
    :param morph:
    :return:
    """

    tokens = nltk.word_tokenize(review)
    normalized_tokens = []
    # normalized_review = ''
    for token in tokens:
        token = morph.parse(token)[0].normal_form
        if token not in stop_words and token not in string.punctuation:
            normalized_tokens.append(token.lower())

    return normalized_tokens

In [63]:
import nltk
nltk.download('wordnet')

from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)



[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\alyona\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [64]:
def prepare_text_for_lda(text):
    tokens = normalize_review(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [65]:
import random
text_data = []
with open('ALL_reviews.txt', encoding='utf-8') as f:
    for line in f:
        tokens = prepare_text_for_lda(line)
        text_data.append(tokens)
        
with open('all_my_reviews.txt', encoding='utf-8') as f:
    for line in f:
        tokens = prepare_text_for_lda(line)            
        text_data.append(tokens)

print('Done!')

Done!


In [66]:
print(text_data[0])

['прекрасный', 'воспоминание', 'вызывать', 'мультфильм', 'детство', 'обожать', 'знать', 'наизусть', 'реплика', 'герой', 'песнь', 'прошлый', 'вырасти', 'мультик', 'любить', 'по-прежнему', 'во-первых', 'отличный', 'яркий', 'картинка', 'нарисовать', 'превосходно', 'передраться', 'правда', 'красивый', 'пейзаж', 'красивый', 'платье', 'маленькая', 'девочка', 'просто', 'завораживать', 'двигаться', 'герой', 'очень', 'реалистично', 'проблема', 'американский', 'аниматор', 'во-вторых', 'озвучка', 'по-моему', 'вариант', 'получиться', 'хороший', 'оригинал', 'огромный', 'спасибо', 'сказать', 'мария', 'живой', 'исполнение', 'саундтрек', 'прекрасный', 'в-третьих', 'герой', 'маленький', 'девочка', 'верить', 'принц', 'просто', 'сразить', 'дмитрий', 'просто', 'воплощение', 'красота', 'постоянно', 'спасать', 'анастасий', 'становиться', 'упомянуть', 'барток', 'просто', 'разряжать', 'обстановка', 'некоторый', 'ситуация', 'единственный', 'пожалуй', 'последний', 'значимость', 'минус', 'фильм', 'нереалистичнос

In [67]:
from gensim import corpora
# Create a corpus from a list of texts
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]
# Сохраним в виде файлов на всякий случай
import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [68]:
import gensim
# Train the model on the corpus.
NUM_TOPICS = 10
ldamodel = gensim.models.ldamodel.LdaModel(
    corpus, 
    num_topics=NUM_TOPICS,
    id2word=dictionary,
)
ldamodel.save('model10.gensim')
topics = ldamodel.print_topics(num_words=15)
for topic in topics:
    print(topic)

(0, '0.019*"фильм" + 0.008*"человек" + 0.006*"самый" + 0.006*"очень" + 0.006*"просто" + 0.005*"герой" + 0.004*"время" + 0.004*"главный" + 0.004*"мультфильм" + 0.004*"история" + 0.004*"большой" + 0.004*"стать" + 0.004*"хороший" + 0.003*"жизнь" + 0.003*"смотреть"')
(1, '0.023*"фильм" + 0.007*"очень" + 0.007*"самый" + 0.006*"просто" + 0.005*"хороший" + 0.005*"жизнь" + 0.005*"время" + 0.005*"герой" + 0.005*"человек" + 0.004*"главный" + 0.004*"история" + 0.004*"персонаж" + 0.004*"картина" + 0.003*"зритель" + 0.003*"большой"')
(2, '0.034*"фильм" + 0.007*"человек" + 0.006*"самый" + 0.005*"хороший" + 0.005*"герой" + 0.004*"просто" + 0.004*"очень" + 0.004*"время" + 0.003*"история" + 0.003*"смотреть" + 0.003*"стать" + 0.003*"жизнь" + 0.003*"большой" + 0.003*"главный" + 0.003*"сказать"')
(3, '0.023*"фильм" + 0.007*"человек" + 0.006*"просто" + 0.006*"герой" + 0.006*"самый" + 0.005*"очень" + 0.005*"хороший" + 0.004*"главный" + 0.004*"сюжет" + 0.004*"персонаж" + 0.004*"время" + 0.003*"сказать" + 0.0

In [ ]:
# dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
# corpus = pickle.load(open('corpus.pkl', 'rb'))
# lda = gensim.models.ldamodel.LdaModel.load('model10.gensim')
# import pyLDAvis.gensim
# lda_display = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary, sort_topics=False)
# pyLDAvis.display(lda_display)

In [80]:
top_topics = ldamodel.top_topics(corpus, topn=15)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / NUM_TOPICS
print('Average topic coherence: %.4f.' % avg_topic_coherence)

Average topic coherence: -0.7886.


In [81]:
from pprint import pprint
pprint(top_topics)

[([(0.034139693, 'фильм'),
   (0.0065164883, 'человек'),
   (0.0056489124, 'самый'),
   (0.005308793, 'хороший'),
   (0.0045773056, 'герой'),
   (0.0041732276, 'просто'),
   (0.00398412, 'очень'),
   (0.003536921, 'время'),
   (0.003456223, 'история'),
   (0.0033295616, 'смотреть'),
   (0.0032991467, 'стать'),
   (0.0032652852, 'жизнь'),
   (0.003195874, 'большой'),
   (0.003051134, 'главный'),
   (0.0029623725, 'сказать')],
  -0.7128169257643693),
 ([(0.022010054, 'фильм'),
   (0.008116831, 'человек'),
   (0.0063568647, 'герой'),
   (0.004976531, 'просто'),
   (0.004810563, 'самый'),
   (0.0046805637, 'жизнь'),
   (0.0042145443, 'хороший'),
   (0.0040954114, 'очень'),
   (0.003646064, 'зритель'),
   (0.0036111737, 'главный'),
   (0.0034486793, 'сюжет'),
   (0.0033981504, 'картина'),
   (0.0032948344, 'время'),
   (0.0030584445, 'сказать'),
   (0.0030016485, 'история')],
  -0.7246044335814794),
 ([(0.022952098, 'фильм'),
   (0.0074658347, 'очень'),
   (0.0072148396, 'самый'),
   (0.005